<a href="https://colab.research.google.com/github/yamagata414/othello/blob/main/nissan_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import heapq
from datetime import datetime, timedelta

In [19]:
locations = {
    "LocationA": (0, 0),
    "LocationB": (5, 0),
    "LocationC": (0, 5),
    "LocationD": (5, 5)
}

ロケーションの設定(辞書)  GPSなどで座標を取得

In [32]:
# ユーザーのリクエストを管理するクラス
class RideRequest:
    def __init__(self, request_id, pickup_location, dropoff_location, pickup_time, max_wait_time):
        self.request_id = request_id                #リクエストの識別子として使われるID
        self.pickup_location = pickup_location      #リクエストの乗車場所を表す文字列やオブジェクト
        self.dropoff_location = dropoff_location    #リクエストの降車場所を表す文字列やオブジェクト
        self.pickup_time = pickup_time              #リクエストの乗車予定時刻を表す datetime オブジェクト
        self.max_wait_time = max_wait_time          #リクエストの最大待ち時間を表す timedelta オブジェクト

初期化　引数(self, リクエストID, 乗車地点, 降車地点, 乗車予定時刻, 最大可能待ち時間)

In [48]:
    def pickup_position(self):                      #乗車地点の座標を返すメソッド
        return locations[self.pickup_location]
    def dropoff_position(self):                     #降車地点の座標を返すメソッド
        return locations[self.dropoff_location]
      #ヒープで比較するためのメソッド
    def __lt__(self, other):
        return self.pickup_time < other.pickup_time

`def __lt__(self, other)`

動作の説明

`self.pickup_time` と `other.pickup_time` は、それぞれ RideRequest オブジェクトの乗車予定時刻を表す
`self.pickup_time < other.pickup_time` は、現在の RideRequest オブジェクトの乗車予定時刻が、他の RideRequest オブジェクトの乗車予定時刻よりも早いかどうかを比較

このメソッドの定義により、自動的に乗車予定時刻の早い順に並べ替えることができる

True を返すと、self の方が other よりも優先順位が高いことを示す

False を返すと、self の方が other よりも後ろの優先順位であることを示す

`__lt__` メソッドを定義することで、Python の組み込み比較演算子 < を用いて、Vehicle オブジェクト同士を自然に比較できる

In [45]:
class Vehicle:     #車両を管理
    def __init__(self, vehicle_id, current_location, available_time, capacity):     #初期化
        self.vehicle_id = vehicle_id                #車両の識別子として使われるID
        self.current_location = current_location    #現在の車両の位置を表す文字列やオブジェクト
        self.available_time = available_time        #車両が次に利用可能になる時刻を表すdatetimeオブジェクト
        self.capacity = capacity                    #車両の収容能力や乗車定員を表す整数値を設定
        self.assigned_requests = []                 #車両に割り当てられたリクエストのリストを初期化

`__init__`引数 (self, 車ID, 現在位置, 次に利用可能になる時間, 車収容人数)
available_time:
リクエストのない車両のavailable_timeは現在時刻,リクエストの割り当て中はリクエストの降車地点に到着する時刻に変更

In [38]:
    #特定のリクエストを車両に割り当てるためのメソッド
    def assign_request(self, request):
        self.assigned_requests.append(request)        #渡されたリクエストを車両の assigned_requests リストに追加　車両がどのリクエストを処理しているかを追跡
        travel_time_to_pickup = self.estimate_travel_time(request.pickup_location)      #車両が現在の位置からリクエストの乗車地点までの移動時間を計算
        travel_time_to_dropoff = self.estimate_travel_time(request.dropoff_location, request.pickup_location)   #リクエストの乗車地点から降車地点までの移動時間を計算  スタートロケーションに、リクエストの乗車地点を指定
        self.available_time = max(self.available_time, request.pickup_time) + timedelta(minutes=travel_time_to_pickup) + timedelta(minutes=travel_time_to_dropoff)      #車両の次の利用可能時間を更新、
        self.current_location = request.dropoff_location    #車両の現在位置を前リクエストの降車位置に変更

 requestを車両に割り当て、乗車および降車地点への移動時間を計算してavailable_timeを更新

` self.available_time = max(self.available_time, request.pickup_time) + timedelta(minutes=travel_time_to_pickup) + timedelta(minutes=travel_time_to_dropoff) `

現在の `available_time` とリクエストの `pickup_time` のうち、遅い方を選択します。これにより、車両が現在のタスクを完了した後、すぐにリクエストに対応できるかどうかが決まります。

例)

- 現在の `available_time`: 10:00
- リクエストの `pickup_time`: 10:15
- `travel_time_to_pickup`: 5 分
- `travel_time_to_dropoff`: 10 分

計算：
1. `max(10:00, 10:15) = 10:15`
2. `10:15 + 5 分 = 10:20`
3. `10:20 + 10 分 = 10:30`

したがって、新しい `available_time` は 10:30 になる


In [39]:
    #実際の距離や交通状況に基づいた移動時間を計算するメソッド
    def estimate_travel_time(self, destination, start_location=None):       #今回は車両の現在位置とリクエストの乗車地点との間の距離を計算し、その距離に基づいて移動時間を推定している  destination目的地 start_location出発地点
        if start_location is None:
            start_location = self.current_location              #引数が1つの時、スタートロケーションを現在の車両の位置にする
        start_coords = locations[start_location]                #座標を代入（辞書locations から取得）
        destination_coords = locations[destination]             #座標を代入
        distance = abs(destination_coords[0] - start_coords[0]) + abs(destination_coords[1] - start_coords[1])  #距離を計算（マンハッタン距離を使用）
        time_in_minutes = distance / 0.2  # 速度0.2での移動時間を計算（分）
        print(f"Distance from {start_location} to {destination} is {distance}")
        print(f"Estimated time is {time_in_minutes} minutes")
        return time_in_minutes                                  #推定移動時間を返す

この関数を考察することでより高度な車両の移動時間を計算することができる

今回は一例として、車両の現在地と、リクエストの乗車予定地の距離をマンハッタン距離で計算し、速度0.2(ブロック/分)で車両の乗車予定地までの移動時間を計算

マンハッタン距離を使用した理由としては、

シンプルで効率的: マンハッタン距離は、計算が単純であり、整数座標上での移動距離を即座に理解できる。さらに速度が一定の場合、移動時間を直接推定することができる

都市型の環境に適している: 車両の移動が都市の格子状の道路網に従う場合、マンハッタン距離は実際の移動時間を比較的正確に表現できるため

In [40]:
    #ヒープで比較するためのメソッド
    def __lt__(self, other):        #車両の利用可能時間に基づいて優先順位付けするために役立つ
        return self.available_time < other.available_time       #Vehicleオブジェクトのavailable_timeと別オブジェクトのavailable_timeを比較しselfが小さいと真を返す

車両の次回利用可能時間基づいて車両を優先順位付けする

True を返すと、self の方が other よりも優先順位が高いことを示す

False を返すと、self の方が other よりも後ろの優先順位であることを示す

`__lt__` メソッドを定義することで、Python の組み込み比較演算子 < を用いて、Vehicle オブジェクト同士を自然に比較できる



In [41]:
def assign_requests_to_vehicles(vehicles, requests):
    for request in requests:
        print(f"\nProcessing request {request.request_id}")     #処理中のリクエストIDを表示
        best_vehicle = None                                     #初期化
        min_travel_time = float('inf')                          #最小の移動距離を追跡するための変数　無限大に初期化

        for vehicle in vehicles:                                #車両を取捨選択する(1台ずつ)
            travel_time_to_pickup = vehicle.estimate_travel_time(request.pickup_location)       #車両の位置とリクエスト乗車一の移動時間を計算
            if vehicle.available_time <= request.pickup_time + request.max_wait_time:           #リクエストの乗車予定時刻と最大待ち可能時間から車両が利用可能か判断
                if travel_time_to_pickup < min_travel_time:                                     #現在の車両の移動時間がこれまでの最小移動時間より短いかを判断
                    min_travel_time = travel_time_to_pickup                                     #車両の移動時間がこれまでの最小移動時間より短いとき、現在の車両の移動時間を最小移動時間に更新
                    best_vehicle = vehicle          #min_travel_time が更新されたとき、best_vehicleを現在の車両に設定する

        if best_vehicle:        #best_vehicleがNoneでないとき
            print(f"Assigning request {request.request_id} to vehicle {best_vehicle.vehicle_id}")   #リクエストがアサインされると車両IDを表示
            best_vehicle.assign_request(request)                #車両にリクエストを割り当てる
        else:
            print(f"No suitable vehicle found for request {request.request_id}")    #最適な車両が見つからないとき表示

    return vehicles     #割り当てが完了した車両のリストを返す

関数の目的:
クエストを適切な車両に割り当てる

処理の向上のためこの関数で車両を利用可能な時間で優先度付けし、最適な車両を選択する際にヒープキューを使用できるようにしている

以下フロー

1. リクエストを処理中であることを表示(3行目)
2. 利用可能な車両の中から、リクエストの乗車予定時刻と最大待ち時間内で利用可能な車両を探す(7～9行目)
3. 各車両の乗車までの移動時間を計算し、最も短い移動時間を持つ車両を選択(10～12行目)
4. 選択された車両にリクエストを割り当て。割り当てられた後、車両の状態（利用可能時間、現在位置など）を更新。(14～16行目)
5. もし最適な車両が見つからない場合、適切なメッセージを表示します。(17、18行目)

In [30]:
# サンプルデータ
vehicles = [
    Vehicle("V1", "LocationA", datetime.now(), 4),
    Vehicle("V2", "LocationB", datetime.now(), 4),
    Vehicle("V3", "LocationC", datetime.now(), 4),
    Vehicle("V4", "LocationD", datetime.now(), 4),
]

requests = [
    RideRequest("R1", "LocationA", "LocationC", datetime.now() + timedelta(minutes=10), timedelta(minutes=15)),
    RideRequest("R2", "LocationB", "LocationD", datetime.now() + timedelta(minutes=20), timedelta(minutes=10)),
    RideRequest("R3", "LocationC", "LocationA", datetime.now() + timedelta(minutes=2), timedelta(minutes=5)),
]

車両とリクエストのサンプルデータ

In [31]:
# 車両の割り当てを実行
assigned_vehicles = assign_requests_to_vehicles(vehicles, requests)

# 結果の出力
print("\nFinal assigned vehicles:")
for vehicle in assigned_vehicles:     #リスト内の各車両オブジェクトに対してループ
    print(f"Vehicle {vehicle.vehicle_id} assigned requests: {[req.request_id for req in vehicle.assigned_requests]}")


Processing request R1
Distance from LocationA to LocationA is 0
Estimated time is 0.0 minutes
Distance from LocationB to LocationA is 5
Estimated time is 25.0 minutes
Distance from LocationC to LocationA is 5
Estimated time is 25.0 minutes
Distance from LocationD to LocationA is 10
Estimated time is 50.0 minutes
Assigning request R1 to vehicle V1
Distance from LocationA to LocationA is 0
Estimated time is 0.0 minutes
Distance from LocationA to LocationC is 5
Estimated time is 25.0 minutes

Processing request R2
Distance from LocationC to LocationB is 10
Estimated time is 50.0 minutes
Distance from LocationB to LocationB is 0
Estimated time is 0.0 minutes
Distance from LocationC to LocationB is 10
Estimated time is 50.0 minutes
Distance from LocationD to LocationB is 5
Estimated time is 25.0 minutes
Assigning request R2 to vehicle V2
Distance from LocationB to LocationB is 0
Estimated time is 0.0 minutes
Distance from LocationB to LocationD is 5
Estimated time is 25.0 minutes

Processi

`assign_requests_to_vehicles` 関数を使って、与えられた車両とリクエストのリストを元に、リクエストを車両に割り当てる
割り当てられた車両のリストが `assigned_vehicles` に格納される(2行目)


assigned_vehicles の各車両について、割り当てられたリクエストの情報を表示(6、7行目)

リクエストのIDはリスト内包表記を使用して取得し、assigned_requests リストから取得
